# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments  \
0  Oct 17 2022  1:20PM  248773        20     8659296   
1  Oct 17 2022  1:17PM  248771        10      129838   
2  Oct 17 2022  1:17PM  248772        10     8659294   
3  Oct 17 2022  1:02PM  248770        10  0086081166   
4  Oct 17 2022  1:02PM  248770        10  0086081167   
5  Oct 17 2022  1:02PM  248770        10  0086081321   
6  Oct 17 2022  1:02PM  248770        10  0086081322   
7  Oct 17 2022  1:02PM  248770        10  0086081347   
8  Oct 17 2022  1:02PM  248770        10  0086081394   
9  Oct 17 2022  1:02PM  248770        10  0086081402   

                         Customer ShipmentStatus  
0  Granules Pharmaceuticals, Inc.       Released  
1                  Virtue Rx, LLC       Released  
2               Yusen – 3D Matrix       Released  
3               ISDIN Corporation       Released  
4               ISDIN Corporation       Released  
5               ISDIN Corporation       Released  
6               ISDIN Corporation       Released  
7               ISDIN Corporation       Released  
8               ISDIN Corporation       Released  
9               ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
22  248769      Executing          1
23  248770       Released          7
24  248771       Released          1
25  248772       Released          1
26  248773       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
248769                NaN        1.0       NaN
248770                NaN        NaN       7.0
248771                NaN        NaN       1.0
248772                NaN        NaN       1.0
248773                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
248702                0.0        0.0       6.0
248706                0.0        7.0       5.0
248707               51.0        4.0       0.0
248719                8.0        3.0       3.0
248734                7.0        0.0       2.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
248702                  0          0         6
248706                  0          7         5
248707                 51          4         0
248719                  8          3         3
248734                  7          0         2

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               248702          0          0         6
1               248706          0          7         5
2               248707         51          4         0
3               248719          8          3         3
4               248734          7          0         2

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               248702                            6
1               248706                   7        5
2               248707        51         4         
3               248719         8         3        3
4               248734         7                  2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Oct 17 2022  1:20PM  248773        20  Granules Pharmaceuticals, Inc.
1   Oct 17 2022  1:17PM  248771        10                  Virtue Rx, LLC
2   Oct 17 2022  1:17PM  248772        10               Yusen – 3D Matrix
3   Oct 17 2022  1:02PM  248770        10               ISDIN Corporation
10  Oct 17 2022  1:01PM  248769        19         GUSA Granules USA, Inc.
11  Oct 17 2022 12:59PM  248768        10   Nextsource Biotechnology, LLC
12  Oct 17 2022 12:46PM  248766        10                     Emerginnova
14  Oct 17 2022 12:25PM  248764        19            Emersa Waterbox, LLC
15  Oct 17 2022 12:24PM  248763        19            Emersa Waterbox, LLC
16  Oct 17 2022 12:15PM  248760        10               ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Oct 17 2022  1:20PM  248773        20  Granules Pharmaceuticals, Inc.   
1  Oct 17 2022  1:17PM  248771        10                  Virtue Rx, LLC   
2  Oct 17 2022  1:17PM  248772        10               Yusen – 3D Matrix   
3  Oct 17 2022  1:02PM  248770        10               ISDIN Corporation   
4  Oct 17 2022  1:01PM  248769        19         GUSA Granules USA, Inc.   
5  Oct 17 2022 12:59PM  248768        10   Nextsource Biotechnology, LLC   
6  Oct 17 2022 12:46PM  248766        10                     Emerginnova   
7  Oct 17 2022 12:25PM  248764        19            Emersa Waterbox, LLC   
8  Oct 17 2022 12:24PM  248763        19            Emersa Waterbox, LLC   
9  Oct 17 2022 12:15PM  248760        10               ISDIN Corporation   

  Completed Executing Released  
0                            1  
1                            1  
2                            1  
3                            7  
4                   1           
5                            1  
6                            2  
7                            1  
8                            1  
9                           24

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Oct 17 2022  1:20PM  248773        20  Granules Pharmaceuticals, Inc.   
1  Oct 17 2022  1:17PM  248771        10                  Virtue Rx, LLC   
2  Oct 17 2022  1:17PM  248772        10               Yusen – 3D Matrix   
3  Oct 17 2022  1:02PM  248770        10               ISDIN Corporation   
4  Oct 17 2022  1:01PM  248769        19         GUSA Granules USA, Inc.   
5  Oct 17 2022 12:59PM  248768        10   Nextsource Biotechnology, LLC   
6  Oct 17 2022 12:46PM  248766        10                     Emerginnova   
7  Oct 17 2022 12:25PM  248764        19            Emersa Waterbox, LLC   
8  Oct 17 2022 12:24PM  248763        19            Emersa Waterbox, LLC   
9  Oct 17 2022 12:15PM  248760        10               ISDIN Corporation   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        7                      
4                  1            
5        1                      
6        2                      
7        1                      
8        1                      
9       24

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 17 2022  1:20PM  248773        20  Granules Pharmaceuticals, Inc.   
1  Oct 17 2022  1:17PM  248771        10                  Virtue Rx, LLC   
2  Oct 17 2022  1:17PM  248772        10               Yusen – 3D Matrix   
3  Oct 17 2022  1:02PM  248770        10               ISDIN Corporation   
4  Oct 17 2022  1:01PM  248769        19         GUSA Granules USA, Inc.   

  Released Executing Completed  
0        1                      
1        1                      
2        1                      
3        7                      
4                  1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 17 2022  1:20PM  248773        20  Granules Pharmaceuticals, Inc.   
1  Oct 17 2022  1:17PM  248771        10                  Virtue Rx, LLC   
2  Oct 17 2022  1:17PM  248772        10               Yusen – 3D Matrix   
3  Oct 17 2022  1:02PM  248770        10               ISDIN Corporation   
4  Oct 17 2022  1:01PM  248769        19         GUSA Granules USA, Inc.   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       7.0        NaN        NaN  
4       NaN        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 17 2022  1:20PM  248773        20  Granules Pharmaceuticals, Inc.   
1  Oct 17 2022  1:17PM  248771        10                  Virtue Rx, LLC   
2  Oct 17 2022  1:17PM  248772        10               Yusen – 3D Matrix   
3  Oct 17 2022  1:02PM  248770        10               ISDIN Corporation   
4  Oct 17 2022  1:01PM  248769        19         GUSA Granules USA, Inc.   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       7.0        0.0        0.0  
4       0.0        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2487525      54.0        7.0        0.0
12          497450       1.0        4.0       51.0
15          497453       5.0        3.0       15.0
16          248750       1.0        0.0        0.0
19         1492557      14.0        1.0        0.0
20          248773       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2487525      54.0        7.0        0.0
1        12   497450       1.0        4.0       51.0
2        15   497453       5.0        3.0       15.0
3        16   248750       1.0        0.0        0.0
4        19  1492557      14.0        1.0        0.0
5        20   248773       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      54.0        7.0        0.0
1        12       1.0        4.0       51.0
2        15       5.0        3.0       15.0
3        16       1.0        0.0        0.0
4        19      14.0        1.0        0.0
5        20       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   54.0
1        12  Released    1.0
2        15  Released    5.0
3        16  Released    1.0
4        19  Released   14.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12    15   16    19   20
Status                                     
Completed   0.0  51.0  15.0  0.0   0.0  0.0
Executing   7.0   4.0   3.0  0.0   1.0  0.0
Released   54.0   1.0   5.0  1.0  14.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12    15   16    19   20
0          Completed   0.0  51.0  15.0  0.0   0.0  0.0
1          Executing   7.0   4.0   3.0  0.0   1.0  0.0
2           Released  54.0   1.0   5.0  1.0  14.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12    15   16    19   20
0  Completed   0.0  51.0  15.0  0.0   0.0  0.0
1  Executing   7.0   4.0   3.0  0.0   1.0  0.0
2   Released  54.0   1.0   5.0  1.0  14.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()